Для первой задачи мы используем данные [Jester Online Joke Recommender System](https://goldberg.berkeley.edu/jester-data/)

**Описание данных**

Файл `train_joke_df.csv` содержит:
- UID - id пользователей
- JID - id шуток, которые 
- Ratin - рейтинг шутки, который проставил пользователь 


Рейтинг имеет значение от -10.00 до 10.00. Могут встречаться значения 99.00, но это обозначает Null (нет рейтинга от пользователя).

Метрика для оценки [RMSE](https://www.codecamp.ru/blog/how-to-interpret-rmse/)

Минимальный RMSE: `4.2238`



In [ ]:
!pip install "scikit-surprise==1.1.3"
!pip install "xlrd==2.0.1"
!pip install optuna -q

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3095451 sha256=5718577cdda82fbaedf22b3c5e6d1e421ae2c7b227d06ee66e106547638ac1f9
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.9/212.9 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 7.1 MB/s eta 0:00:00


### Import

In [ ]:
import numpy as np
import pandas as pd
from collections import defaultdict
from surprise import Dataset, Reader, KNNWithMeans, accuracy, SVD, SVDpp, NMF, SlopeOne, KNNWithZScore
from surprise.model_selection import GridSearchCV
from surprise.model_selection import train_test_split
from sklearn.model_selection import train_test_split as tts
from surprise.model_selection import KFold
import optuna

from google.colab import drive
drive.mount('/content/drive')

np.random.seed(42)

Mounted at /content/drive


### Базовые функции для скоринга и получения рекомендаций

In [ ]:
def get_num_user_ratings(uid):
    """ возвращает кол-во рейтингов у пользователя 
    args: 
      uid: id пользователей
    returns: 
      кол-во объектов, которые оценил пользователь
    """
    try:
        return len(trainset.ur[trainset.to_inner_uid(uid)])
    except ValueError: # пользователя не было во время обучения (новый, отправить на стартовые рекомендации)
        return 0
    
def get_num_item_ratings(iid):
    """ возвращает кол-во пользователей, которые оценили выбранный элемент 
    args:
      iid: строка с элементов рекомендации
    returns:
      кол-во пользователей, которые дали оценки по элементу
    """
    try: 
        return len(trainset.ir[trainset.to_inner_iid(iid)])
    except ValueError:
        return 0
    
# На основе Surprise FAQ построим рекомендации Топ-N
def get_top_n(predictions, n=5):
    """Определят Топ-N рекомендаций

    Args:
        predictions(list of Prediction objects): Списко рекомендаций, из алгоритма Surprise
        n(int): Кол-во топ рекомендаций

    Returns:
        Словарь пользователь - список рекомендакиций для пользователей
        [(raw item id, rating estimation), ...]
    """

    # Предикт для каждого пользователя
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Сортировка предикта (по пользователям)
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n    

### Загрузка и обработка данных

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Bootcamp_Магнит/train_joke_df.csv')

df.head(5)

,UID,JID,Rating
0,18029,6,-1.26
1,3298,64,-4.17
2,3366,58,0.92
3,12735,92,3.69
4,11365,38,-6.60


In [ ]:
# сделаем сортировку и перепишем index
df = df.sort_values(by=['UID', 'JID'])
df = df.reset_index(drop=True)

In [ ]:
# создадим на основе набора данных
# поднабор, который требуется для библиотеки Surprise

# указываем минимальный и максимальный рейтинги
reader = Reader(rating_scale=(-10, 10))

# передаём набор, указывая последовательность колонок: user (raw) ids, item (raw) ids, ratings
# для Surprise - это обязательно
data = Dataset.load_from_df(df[['UID', 'JID', 'Rating']], reader)

In [ ]:
trainset_data = data.build_full_trainset()

# сделаем разделение на обучающую и тестовую выборку
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

### Обучение модели

In [ ]:
# Подберём гиперпараметры

def objective_SVD(trial):    
    reg_pu = trial.suggest_float("reg_pu", 1e-6, 0.01, log=True)
    reg_qi = trial.suggest_float("reg_qi", 0.01, 1, log=True)
    reg_bu = trial.suggest_float("reg_bu", 1e-7, 0.01, log=True)
    reg_bi = trial.suggest_float("reg_bi", 1e-7, 0.01, log=True)
    
    lr_pu = trial.suggest_float("lr_pu", 0.0001, 0.01, log=True)
    lr_qi = trial.suggest_float("lr_qi", 0.0001, 0.01, log=True)
    lr_bu = trial.suggest_float("lr_bu", 0.0001, 0.01, log=True)
    lr_bi = trial.suggest_float("lr_bi", 0.0001, 0.01, log=True)
    

    #reg_all = trial.suggest_float("reg_all", 1e-5, 2, log=True)
    #lr_all = trial.suggest_float("lr_all", 0.0001, 0.01, log=True)
    n_factors = trial.suggest_int("n_factors", 100, 2000)
    n_epochs = trial.suggest_int("n_epochs", 10, 50)

    #init_mean = trial.suggest_float("init_mean", 0, 10, log=False)
    #init_std_dev = trial.suggest_float("init_std_dev", 0.01, 2, log=True)
    
    kf = KFold(n_splits=3)

    algo = SVD(reg_pu=reg_pu, reg_qi=reg_qi, reg_bu=reg_bu, reg_bi=reg_bi,
               lr_pu=lr_pu, lr_qi=lr_qi, lr_bu=lr_bu, lr_bi=lr_bi,
               n_factors=n_factors, n_epochs=n_epochs)

    rmse = []
    
    for trainset, testset in kf.split(data):

      algo.fit(trainset)
      predictions = algo.test(testset)

      rmse.append(accuracy.rmse(predictions))

    score = np.array(rmse).mean()

    return score


study = optuna.create_study(direction="minimize")
study.optimize(objective_SVD, n_trials=40)

[I 2023-04-26 09:18:06,993] A new study created in memory with name: no-name-a88de648-8763-4bab-9283-138289238824


RMSE: 4.0683
RMSE: 4.0696
RMSE: 4.0702


[I 2023-04-26 09:21:08,885] Trial 0 finished with value: 4.069347680962569 and parameters: {'reg_pu': 1.220855211403125e-06, 'reg_qi': 0.5478595913941109, 'reg_bu': 0.00780509079637674, 'reg_bi': 0.0012921586468483987, 'lr_pu': 0.009294438389589607, 'lr_qi': 0.00019683272170000586, 'lr_bu': 0.0005726084296876974, 'lr_bi': 0.00018818090671546834, 'n_factors': 640, 'n_epochs': 13}. Best is trial 0 with value: 4.069347680962569.


RMSE: 4.1962
RMSE: 4.1943
RMSE: 4.1973


[I 2023-04-26 09:36:44,688] Trial 1 finished with value: 4.195945833771931 and parameters: {'reg_pu': 0.0019942266906958377, 'reg_qi': 0.037036423070412106, 'reg_bu': 0.00019715290218143904, 'reg_bi': 5.490703496667441e-06, 'lr_pu': 0.00039099856905620763, 'lr_qi': 0.00016099608012160935, 'lr_bu': 0.0008818558544526172, 'lr_bi': 0.0036613617252923927, 'n_factors': 1839, 'n_epochs': 34}. Best is trial 0 with value: 4.069347680962569.


...

[I 2023-04-26 13:21:09,277] Trial 37 finished with value: 4.057295637723957 and parameters: {'reg_pu': 0.0011448591457074886, 'reg_qi': 0.8440497048460534, 'reg_bu': 6.467071058997201e-05, 'reg_bi': 0.009931275642834672, 'lr_pu': 0.004968717128433608, 'lr_qi': 0.00029736607033096607, 'lr_bu': 0.0007493116095528266, 'lr_bi': 0.00017629806864810502, 'n_factors': 1072, 'n_epochs': 36}. Best is trial 34 with value: 4.056740405924985.


RMSE: 4.0622
RMSE: 4.0572
RMSE: 4.0570


[I 2023-04-26 13:32:43,295] Trial 38 finished with value: 4.058804900798624 and parameters: {'reg_pu': 0.0022192574554651193, 'reg_qi': 0.8788468775231728, 'reg_bu': 6.608676746683275e-05, 'reg_bi': 0.007097475449305056, 'lr_pu': 0.005231319116966904, 'lr_qi': 0.0001408221010353754, 'lr_bu': 0.0014584506966893265, 'lr_bi': 0.00016119034201014532, 'n_factors': 1177, 'n_epochs': 37}. Best is trial 34 with value: 4.056740405924985.


RMSE: 4.0568
RMSE: 4.0497
RMSE: 4.0567


[I 2023-04-26 13:46:49,912] Trial 39 finished with value: 4.054419398620368 and parameters: {'reg_pu': 0.0010398462453200708, 'reg_qi': 0.975481537099458, 'reg_bu': 6.405279534313364e-05, 'reg_bi': 0.008446937909973943, 'lr_pu': 0.005037130407846973, 'lr_qi': 0.0001532653071418146, 'lr_bu': 0.0016783726294656905, 'lr_bi': 0.00014842221255242954, 'n_factors': 1235, 'n_epochs': 44}. Best is trial 39 with value: 4.054419398620368.


In [ ]:
# Kaggle Score = 3.97333

# обучим с лучшими параметрами (Trial 39 из ячейки выше)
params = {'reg_pu': 0.0010398462453200708, 'reg_qi': 0.975481537099458, 'reg_bu': 6.405279534313364e-05, 'reg_bi': 0.008446937909973943, 'lr_pu': 0.005037130407846973, 'lr_qi': 0.0001532653071418146, 'lr_bu': 0.0016783726294656905, 'lr_bi': 0.00014842221255242954, 'n_factors': 1235, 'n_epochs': 44}

algo = SVD(reg_pu=params['reg_pu'], reg_qi=params['reg_qi'], reg_bu=params['reg_bu'], reg_bi=params['reg_bi'], 
           lr_pu=params['lr_pu'], lr_qi=params['lr_qi'], lr_bu=params['lr_bu'], lr_bi=params['lr_bi'],
            n_factors=params['n_factors'], n_epochs=params['n_epochs'])
algo.fit(trainset)

# получим предикт и посмотрим метрику
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 4.0176


4.017592166765312

In [ ]:
# обучим с лучшими параметрами на всём трейне
algo.fit(trainset_data)

### Тестирование и результаты

In [ ]:
# получаем предикт
uid = 1  # id пользователя 
iid = 1  # iв шутки

# получим предик на основе обученных данных
# -7.82 - это фактический рейтинг, но посмотрим, какой ответ будет в предикте
pred = algo.predict(uid, iid, r_ui=-7.82, verbose=True)

user: 1          item: 1          r_ui = -7.82   est = -7.49   {'was_impossible': False}


In [ ]:
uid = 24983  # id пользователя 
iid = 62     # iв шутки

pred = algo.predict(uid, iid, r_ui=-0.29, verbose=True)

user: 24983      item: 62         r_ui = -0.29   est = -0.05   {'was_impossible': False}


### Обзор рекомендаций

In [ ]:
# построим таблицу для обзора набора рекомендаций
# посмотрим, какие элементы и в каком кол-ве рекомендуем
trainset = algo.trainset

predictions_df = pd.DataFrame(predictions, columns=['uid', 'iid', 'rui', 'est', 'details'])

predictions_df['№ кол-во пользовательских рейтингов'] = predictions_df.uid.apply(get_num_user_ratings)
predictions_df['№ кол-во рейтингов элементов'] = predictions_df.iid.apply(get_num_item_ratings)
predictions_df['error'] = abs(predictions_df.est - predictions_df.rui)

best_predictions = predictions_df.sort_values(by='error')[:10]
worst_predictions = predictions_df.sort_values(by='error')[-10:]

In [ ]:
best_predictions.head(5)

,uid,iid,rui,est,details,№ кол-во пользовательских рейтингов,№ кол-во рейтингов элементов,error
19327,2889,78,6.12,6.120016,{'was_impossible': False},73,7194,0.000016
19089,15536,26,0.00,-0.000017,{'was_impossible': False},56,18969,0.000017
254695,18062,81,-3.74,-3.740034,{'was_impossible': False},70,7437,0.000034
62939,21634,62,2.33,2.330067,{'was_impossible': False},59,19985,0.000067
156685,7692,47,4.76,4.759929,{'was_impossible': False},54,17852,0.000071


In [ ]:
# Предикт для всех, кого нет в выборке для обучения
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions)

# Сделаем вывод рекомендаций
a=0
for uid, user_ratings in top_n.items():
    a+=1
    print(uid, [iid for (iid, _) in user_ratings])
    
    if a==10:
        break

19208 [54, 55, 29, 78, 89]
8671 [8, 83, 77, 32, 72]
6037 [78, 81, 89, 77, 87]
3233 [50, 36, 27, 31, 42]
3449 [100, 80, 6, 73, 53]
10032 [35, 94, 32, 53, 49]
5774 [89, 47, 32, 68, 29]
23392 [36, 48, 29, 53, 21]
3039 [62, 53, 94, 91, 96]
17395 [27, 88, 50, 35, 85]


### Для отправки на тестирование

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/Bootcamp_Магнит/test_joke_df_nofactrating.csv', index_col=0)
test.head(5)

,UID,JID
InteractionID,,
0,11228,39
1,21724,85
2,16782,56
3,12105,42
4,14427,2


In [ ]:
test['Rating'] = test[['UID', 'JID']].apply(lambda x: algo.predict(x[0], x[1], verbose=False).est,
                                                      axis = 1)

In [ ]:
# вид набора данных, который должен быть отправлен для тестирования
test['Rating'].to_frame().head(5)

,Rating
InteractionID,
0,3.072118
1,-7.812274
2,-0.214448
3,7.111174
4,6.230477


In [ ]:
# формирование файла для отправки в Kaggle
test['Rating'].to_frame().to_csv('/content/drive/MyDrive/Bootcamp_Магнит/Chernov_Klim_nn.csv')